In [32]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, ClassLabel, DatasetDict
import numpy as np

import math
import time
from typing import Optional, Tuple, Dict
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import f1_score

In [2]:
dataset = load_dataset("Densu341/Fresh-rotten-fruit")

In [3]:
remove_labels = [18, 20, 16, 13, 2, 5, 7, 9]
labels = np.array(dataset["train"]["label"])
mask = ~np.isin(labels, remove_labels)

# 3. 필요 없는 라벨 제거
clean_dataset = dataset["train"].select(np.where(mask)[0])

# 4. train/val split
dataset = clean_dataset.train_test_split(test_size=0.2)
train_dataset, val_dataset = dataset["train"], dataset["test"]

# 5. 실제 남은 라벨 인덱스 및 이름 추출
unique_labels = sorted(set(train_dataset["label"]) | set(val_dataset["label"]))
all_labels = [train_dataset.features["label"].int2str(i) for i in unique_labels]

# 6. 새로운 ClassLabel 정의
new_classlabel = ClassLabel(num_classes=len(all_labels), names=all_labels)

# 7. 라벨 값 재매핑
def remap_labels(example):
    label_name = train_dataset.features["label"].int2str(example["label"])
    example["label"] = all_labels.index(label_name)
    return example

train_dataset = train_dataset.map(remap_labels)
val_dataset   = val_dataset.map(remap_labels)

train_dataset = train_dataset.cast_column("label", new_classlabel)
val_dataset   = val_dataset.cast_column("label", new_classlabel)

# 8. 최종 DatasetDict 생성
final_dataset = DatasetDict({
    "train": train_dataset,
    "test": val_dataset
})

Map:   0%|          | 0/21486 [00:00<?, ? examples/s]

Map:   0%|          | 0/5372 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/21486 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/5372 [00:00<?, ? examples/s]

In [28]:
class Cfg:
    IMAGE_SIZE = 224  # (H,W) 전처리에서 Resize((224,224)) 가정
    PATCH_DOWN = 16   # CNN으로 1/16 해상도까지 다운샘플 (224->14)
    D_MODEL = 384     # Transformer 임베딩 차원
    N_HEAD = 6
    DEPTH  = 6
    MLP_RATIO = 4
    DROPOUT = 0.1
    ATTN_DROPOUT = 0.1

    # 학습 관련
    LR = 3e-4
    EPOCHS = 10
    BATCH_SIZE = 32
    NUM_WORKERS = 2
    #쥬피터 노트북에선 2이상 안됨
    MIXED_PRECISION = True

    # 태스크 토글
    MULTITASK = False  # True: (과일종, 신선도) 두 개의 헤드
    NUM_CLASSES = 14   # 단일 과제일 때 클래스 수

    # 멀티태스크일 때 각 헤드 클래스 수
    NUM_FRUIT = 7
    NUM_FRESH = 2

In [29]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])  # ImageNet 기준
])

val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

# --------------------------------------------------
# PyTorch Dataset 래퍼
# --------------------------------------------------
class FruitHFDataset(Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        image = sample["image"].convert("RGB")  # RGBA → RGB
        label = sample["label"]                # 이미 0~N-1 정수
        if self.transform:
            image = self.transform(image)
        return image, label

# --------------------------------------------------
# DataLoader 생성
# --------------------------------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


train_loader = DataLoader(
    FruitHFDataset(final_dataset["train"], transform=train_transform),
    batch_size=Cfg.BATCH_SIZE,         # 보통 32~64
    shuffle=True,
    drop_last=True,
    num_workers=Cfg.NUM_WORKERS,       # Cfg에서 관리 (예: 1)
    pin_memory=(device.type == "cuda") # ✅ GPU일 때만 True
)

val_loader = DataLoader(
    FruitHFDataset(final_dataset["test"], transform=val_transform),
    batch_size=Cfg.BATCH_SIZE,
    shuffle=False,
    num_workers=Cfg.NUM_WORKERS,
    pin_memory=(device.type == "cuda") # ✅ 동일하게 적용 가능
)

num_classes = len(final_dataset["train"].features["label"].names)

In [8]:
# 1. 고유 라벨 추출
unique_labels = sorted(set(final_dataset["train"]["label"]) | set(final_dataset["test"]["label"]))
print("실제 라벨 인덱스:", unique_labels)

# 2. 재매핑 딕셔너리 생성
label2idx = {old: new for new, old in enumerate(unique_labels)}
print("라벨 매핑:", label2idx)

# 3. 매핑 함수
def remap(example):
    example["label"] = label2idx[example["label"]]
    return example

# 4. 라벨 다시 매핑
final_dataset = final_dataset.map(remap)

# 5. 클래스 개수 확인
num_classes = len(unique_labels)
print("최종 클래스 개수:", num_classes)


실제 라벨 인덱스: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
라벨 매핑: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13}


Map:   0%|          | 0/21486 [00:00<?, ? examples/s]

Map:   0%|          | 0/5372 [00:00<?, ? examples/s]

최종 클래스 개수: 14


In [30]:
# =========================
# 간단 CNN 베이스라인 (SeparableConv)
# - 과대적합 억제: Depthwise Separable, Dropout, GAP
# - 멀티태스크/단일과제 공통 지원 (기존 train_loop 그대로 사용 가능)
# =========================

class SeparableConv(nn.Module):
    def __init__(self, cin, cout, stride=1, drop=0.0):
        super().__init__()
        self.dw = nn.Conv2d(cin, cin, kernel_size=3, stride=stride, padding=1, groups=cin, bias=False)
        self.dw_bn = nn.BatchNorm2d(cin)
        self.pw = nn.Conv2d(cin, cout, kernel_size=1, bias=False)
        self.pw_bn = nn.BatchNorm2d(cout)
        self.act = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout2d(drop) if drop > 0 else nn.Identity()

    def forward(self, x):
        x = self.dw(x); x = self.dw_bn(x); x = self.act(x)
        x = self.pw(x); x = self.pw_bn(x); x = self.act(x)
        x = self.dropout(x)
        return x

class SmallCNN(nn.Module):
    """
    입력: (B,3,224,224) 가정
    다운샘플: /2 → /2 → /2 → /2 → /2 (총 /32, 224→7)
    채널: 32 → 64 → 128 → 192 → 256 (가벼움)
    분류: GAP → Dropout → Linear
    """
    def __init__(self,
                 num_classes: Optional[int] = None,
                 multitask: bool = False,
                 num_fruit: int = 7,
                 num_fresh: int = 2,
                 drop_block: float = 0.05,
                 drop_head: float = 0.2):
        super().__init__()
        self.multitask = multitask

        chs = [32, 64, 128, 192, 256]  # 가벼운 채널 구성
        self.stem = nn.Sequential(
            nn.Conv2d(3, chs[0], kernel_size=3, stride=2, padding=1, bias=False),  # 224 -> 112
            nn.BatchNorm2d(chs[0]),
            nn.ReLU(inplace=True),
        )
        self.stage1 = SeparableConv(chs[0], chs[1], stride=2, drop=drop_block)     # 112 -> 56
        self.stage2 = SeparableConv(chs[1], chs[2], stride=2, drop=drop_block)     # 56  -> 28
        self.stage3 = SeparableConv(chs[2], chs[3], stride=2, drop=drop_block)     # 28  -> 14
        self.stage4 = SeparableConv(chs[3], chs[4], stride=2, drop=drop_block)     # 14  -> 7

        self.gap = nn.AdaptiveAvgPool2d(1)
        self.head_drop = nn.Dropout(p=drop_head)

        feat_dim = chs[-1]
        if multitask:
            self.head_fruit = nn.Linear(feat_dim, num_fruit)
            self.head_fresh = nn.Linear(feat_dim, num_fresh)
        else:
            assert num_classes is not None, "num_classes 를 지정하세요 (단일 과제 모드)."
            self.head = nn.Linear(feat_dim, num_classes)

        # 간단 초기화
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity="relu")
            elif isinstance(m, nn.Linear):
                nn.init.trunc_normal_(m.weight, std=0.02)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)              # (B, C, 7, 7)
        x = self.gap(x).squeeze(-1).squeeze(-2)  # (B, C)
        x = torch.flatten(x, 1)
        x = self.head_drop(x)

        if self.multitask:
            return self.head_fruit(x), self.head_fresh(x)
        else:
            return self.head(x), None
        


In [ ]:
torch.backends.cudnn.benchmark = True

# ==== AMP on/off를 디바이스에 맞춰 안전하게 ====
def amp_enabled(device: torch.device) -> bool:
    return getattr(Cfg, "MIXED_PRECISION", True) and device.type == "cuda"


def loss_single(logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
    return F.cross_entropy(logits, targets)

def loss_multi(
    logits_fruit: torch.Tensor, targets_fruit: torch.Tensor,
    logits_fresh: torch.Tensor, targets_fresh: torch.Tensor,
    alpha: float = 1.0, beta: float = 1.0
) -> torch.Tensor:
    lf = F.cross_entropy(logits_fruit, targets_fruit)
    ls = F.cross_entropy(logits_fresh, targets_fresh)
    return alpha * lf + beta * ls

def _get_logits(outputs):
    return outputs[0] if isinstance(outputs, (tuple, list)) else outputs


def train_loop(model: nn.Module,
               train_loader: torch.utils.data.DataLoader,
               val_loader: Optional[torch.utils.data.DataLoader],
               optimizer: torch.optim.Optimizer,
               epochs: int,
               device: torch.device,
               multitask: bool = False,
               scheduler: Optional[torch.optim.lr_scheduler._LRScheduler] = None,
               grad_clip: Optional[float] = None):
    
    if device.type == "cuda":
        gpu_name = torch.cuda.get_device_name(device)
        print(f" Using GPU: {gpu_name}")
        print(f"    CUDA capability: {torch.cuda.get_device_capability(device)}")
        print(f"    Memory: {torch.cuda.get_device_properties(device).total_memory / 1024**3:.1f} GB\n")
    else:
        print("  Using CPU (no CUDA available)\n") #확인용 코드임

    ce = nn.CrossEntropyLoss()
    scaler = torch.cuda.amp.GradScaler(enabled=amp_enabled(device))

    for epoch in range(1, epochs + 1):
        model.train()
        t0 = time.time()
        running_loss = 0.0
        running_correct, running_total = 0, 0

        loop = tqdm(train_loader, desc=f"[Train] {epoch}/{epochs}", leave=True)
        for batch in loop:
            optimizer.zero_grad(set_to_none=True)

            if multitask:
                images, (y_fruit, y_fresh) = batch
                images = images.to(device, non_blocking=True)
                y_fruit = y_fruit.to(device, non_blocking=True).long()
                y_fresh = y_fresh.to(device, non_blocking=True).long()

                with torch.cuda.amp.autocast(enabled=amp_enabled(device)):
                    logits_fruit, logits_fresh = model(images)
                    loss = ce(logits_fruit, y_fruit) + ce(logits_fresh, y_fresh)

                preds = logits_fruit.argmax(dim=1)
                running_correct += (preds == y_fruit).sum().item()
                running_total   += y_fruit.size(0)

            else:
                images, y = batch
                images = images.to(device, non_blocking=True)
                y = y.to(device, non_blocking=True).long()

                with torch.cuda.amp.autocast(enabled=amp_enabled(device)):
                    outputs = model(images)
                    logits  = _get_logits(outputs)
                    loss    = ce(logits, y)

                preds = logits.argmax(dim=1)
                running_correct += (preds == y).sum().item()
                running_total   += y.size(0)

            scaler.scale(loss).backward()
            if grad_clip is not None:
                scaler.unscale_(optimizer)
                nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            scaler.step(optimizer)
            scaler.update()

            running_loss += float(loss.detach().cpu())
            loop.set_postfix(loss=f"{loss.item():.4f}",
                             lr=f"{optimizer.param_groups[0]['lr']:.2e}")

@torch.no_grad()
def evaluate(model: nn.Module,
             loader: torch.utils.data.DataLoader,
             device: torch.device,
             multitask: bool = False) -> Dict[str, float]:
    model.eval()
    ce = nn.CrossEntropyLoss()
    total_loss = 0.0

    all_preds = []
    all_labels = []

    if multitask:
        # 멀티태스크의 경우 과일 헤드만 F1 계산
        for images, (y_fruit, y_fresh) in loader:
            images = images.to(device, non_blocking=True)
            y_fruit = y_fruit.to(device, non_blocking=True).long()
            y_fresh = y_fresh.to(device, non_blocking=True).long()

            logits_fruit, logits_fresh = model(images)
            loss = ce(logits_fruit, y_fruit) + ce(logits_fresh, y_fresh)
            total_loss += float(loss.detach().cpu())

            preds = logits_fruit.argmax(dim=1).cpu().numpy()
            labels = y_fruit.cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels)

        f1 = f1_score(all_labels, all_preds, average='macro')  # macro F1
        return {"val_loss": total_loss / max(1, len(loader)),
                "val_f1": f1}

    else:
        # 단일 과제 (다중클래스 분류)
        for images, y in loader:
            images = images.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True).long()

            outputs = model(images)
            logits = _get_logits(outputs)
            loss = ce(logits, y)
            total_loss += float(loss.detach().cpu())

            preds = logits.argmax(dim=1).cpu().numpy()
            labels = y.cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels)

        f1 = f1_score(all_labels, all_preds, average='macro')  # 다중클래스용 macro
        return {"val_loss": total_loss / max(1, len(loader)),
                "val_f1": f1}

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Cfg.MULTITASK   = False
Cfg.NUM_CLASSES = 14

model = SmallCNN(num_classes=Cfg.NUM_CLASSES, multitask=False).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=Cfg.LR, weight_decay=5e-4)

train_loop(model, train_loader, val_loader, optimizer, Cfg.EPOCHS, device,
           multitask=False, scheduler=None, grad_clip=1.0)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_5344\3923261510.py:43: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=amp_enabled(device))


🚀 Using GPU: NVIDIA GeForce RTX 4060 Laptop GPU
    CUDA capability: (8, 9)
    Memory: 8.0 GB



[Train] 1/10:   0%|          | 0/671 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
fruit_names = ["사과", "바나나", "토마토", "배", "포도"]  # 추후 수정
fresh_status = ["신선", "상함"]

def predict(model, image):
    model.eval()
    with torch.no_grad():
        image = image.to(device).unsqueeze(0)  # [1, C, H, W]
        outputs = model(image)
        pred = torch.argmax(outputs, dim=1).item()

        fruit_idx = pred // 2
        status_idx = pred % 2
        return f"{fruit_names[fruit_idx]} - {fresh_status[status_idx]}"